In [7]:
import os
!apt-get update -qq > /dev/null   
!apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
!pip install nlu  pyspark==2.4.7 > /dev/null   
!pip install flask_ngrok
import nlu

In [8]:
API = "http://ec2-3-108-196-152.ap-south-1.compute.amazonaws.com:5000/api/"

In [31]:
from flask import Flask,request,jsonify
from flask_ngrok import run_with_ngrok
import json
import math
import requests

app = Flask(__name__)
run_with_ngrok(app)

multi_pipe = None
col_names = ['sentence_embedding_bert','sentence_embedding_electra', 'sentence_embedding_use']

@app.route("/load-api")
def loadAPI():
    try:
        global multi_pipe
        multi_pipe = nlu.load('use en.embed_sentence.electra embed_sentence.bert')
        return {"status" : 200},200
    except:
        return {"status" : 500},500

@app.route("/generate-predictions",methods = ['GET', 'POST'])
def genearatePredictions():

    title = request.form['title']
    
    if title and multi_pipe:
        predictions = multi_pipe.predict(title,get_embeddings=True).iloc[0]
        
        response = {}

        for col_name in col_names:
            response[col_name] = list(predictions[col_name][0])
        
        return { "predictions" : response ,"status" : 201},201
    else:
        return { "error" :"ERROR" ,"status" : 500}, 500

def dot(A,B): 
    return (sum(a*b for a,b in zip(A,B)))

def cosine_similarity(a,b):
    return dot(a,b) / ( math.sqrt(dot(a,a) * dot(b,b)) )

@app.route("/get-similar-questions",methods=['GET','POST'])
def getSimilarQuestions():
    data = requests.get(API+"discussion/predictions").json()
    embeddings = eval(request.form['predictions'])
    print(embeddings)
    if data['status'] == 200 and embeddings:
        predictions = data['predictions']
        for i in range(len(predictions)):
            similarity = 0
            for e_col in col_names:
                if e_col in predictions[i]['predictions'] and e_col in embeddings:
                    similarity += cosine_similarity(predictions[i]['predictions'][e_col],embeddings[e_col])

            predictions[i]['similarity_score'] = similarity/len(col_names)
        
        return {"similarQuestions" : sorted(predictions,key = lambda x : x['similarity_score'],reverse=True)[:3] , "status" : 201} , 201
    else:
        return { "status" : 500,"error" :"ERROR"}, 500

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://4a60-34-80-212-55.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
sent_electra_small_uncased download started this may take some time.
Approximate size to download 48.7 MB
[OK!]
sent_small_bert_L2_128 download started this may take some time.
Approximate size to download 16.1 MB
[OK!]
sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]

127.0.0.1 - - [27/Nov/2021 06:47:06] "GET /load-api HTTP/1.1" 200 -


127.0.0.1 - - [27/Nov/2021 06:47:12] "POST /get-similar-questions HTTP/1.1" 201 -


{'sentence_embedding_bert': [-1.673563838005066, 0.2529952824115753, 0.11113935708999634, -1.4586787223815918, -0.23732517659664154, 0.18339455127716064, -1.0539880990982056, 1.0314900875091553, -1.1652895212173462, -1.0171369314193726, 0.5875211358070374, 0.19714407622814178, 0.3699447810649872, 0.2824907600879669, 1.5939723253250122, -0.9041640162467957, -1.3685723543167114, 0.4786406457424164, -0.1594671607017517, 0.5731086134910583, -1.2294644117355347, -0.12459596246480942, 2.1996071338653564, 0.19090807437896729, 0.8145908713340759, -0.14642710983753204, 0.6407742500305176, 1.042299747467041, 0.01346455980092287, -1.173380732536316, -0.6126505732536316, -2.145663022994995, -1.1649717092514038, -0.25048354268074036, 0.899552047252655, 0.7314236164093018, 0.6890482306480408, -0.311556339263916, -1.478550910949707, -0.6361036896705627, 1.1887954473495483, 0.00589798903092742, 0.29506975412368774, -1.070624589920044, 0.3362205922603607, -1.6387754678726196, -1.269498348236084, 0.2403

127.0.0.1 - - [27/Nov/2021 06:47:49] "POST /generate-predictions HTTP/1.1" 201 -
127.0.0.1 - - [27/Nov/2021 06:48:11] "POST /get-similar-questions HTTP/1.1" 201 -


{'sentence_embedding_bert': [-1.1666247844696045, 0.6217620968818665, -0.423587441444397, -1.7175302505493164, 0.5490631461143494, 0.025017591193318367, -0.6362046599388123, 0.6648552417755127, -0.765775203704834, 0.27949389815330505, 0.3022098243236542, 0.24967563152313232, -1.6937259435653687, 0.8555940985679626, 2.1871745586395264, -1.599981665611267, -0.30640649795532227, 0.032449837774038315, -1.8453397750854492, 0.7820696234703064, -0.7649328112602234, -0.6296379566192627, 2.569363594055176, 0.23302657902240753, 1.1615134477615356, 0.8651819825172424, -0.4241826832294464, 1.5087605714797974, 0.881505012512207, -0.618385374546051, -0.6293026208877563, -1.2141821384429932, -0.6884154677391052, -0.7597681879997253, 0.9554076194763184, -0.8667950630187988, -0.1588595062494278, -0.4356202185153961, -2.2637243270874023, -0.8480132222175598, 0.882178008556366, -0.5343073606491089, 1.1594023704528809, -0.9973373413085938, 0.9307770729064941, -1.3873745203018188, -0.9729602932929993, -0.6